**Задача поиск похожих по эмбеддингам**
Скачиваем датасет 

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

```
# пример работы с ним 
from corus import load_ods_gazeta
path = 'gazeta.csv.gz'
records = load_ods_gazeta(path)
next(records)
```


что надо сделать 

1. на основе word2vec/fasttext реализовать метод поиска ближайших статей
(на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2. Проверить насколько хорошо работают подходы

!pip install corus pymorphy2 stop-words annoy gensim

Так как датасет газеты недоступен, по рекомендации преподавателя будем использовать датасет новостей майлру

!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [ ]:
from corus import load_lenta

path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)


In [8]:
import pandas as pd
import numpy as np

import gensim.downloader as api
from gensim.models import Word2Vec, FastText
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string
from sklearn.model_selection import train_test_split
import annoy
import re
from tqdm import tqdm
from tqdm import tqdm_notebook

import gensim.models
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [9]:
corpus_title = []
corpus_text = []

for num, iter in tqdm(enumerate(records)):
    corpus_title.append(iter.title)
    corpus_text.append(iter.text)
    if num == 99999:
        break

99999it [00:03, 26057.56it/s]


In [10]:
df = pd.DataFrame(corpus_title, columns=['title'])
df['text'] = corpus_text
df.head()

,title,text
0,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...
1,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...
2,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...
3,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...
4,Архиепископ канонической УПЦ отказался прийти ...,Архиепископ канонической Украинской православн...


In [11]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [12]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [13]:
pattern = r'[^а-яА-Я0-9]'
df['text'] = df['text'].apply(lambda x: re.sub(pattern, ' ', x))

In [14]:
sentences = []
counter = 0

for line in tqdm_notebook(df.text):
    spls = preprocess_txt(line)
    sentences.append(spls)
    counter += 1      
    if counter > 100000:
        break

  0%|          | 0/100000 [00:00<?, ?it/s]

In [18]:
%%time
modelW2V = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=2)

CPU times: user 2min 7s, sys: 660 ms, total: 2min 8s
Wall time: 45.9 s


In [19]:
%%time
modelFT = FastText(sentences=sentences, vector_size=100, min_count=2, window=5, workers=8)

CPU times: user 18min 23s, sys: 4.82 s, total: 18min 28s
Wall time: 2min 36s


In [20]:
w2v_index = annoy.AnnoyIndex(100 ,'angular')
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

for line in tqdm_notebook(df.text):
        n_w2v = 0
        n_ft = 0
        spls = line
        index_map[counter] = spls
        question = preprocess_txt(spls)
        
        vector_w2v = np.zeros(100)
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        if counter > 100000:
            break
    

w2v_index.build(10)
ft_index.build(10)

  0%|          | 0/100000 [00:00<?, ?it/s]

True

In [21]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(100)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

In [22]:
TEXT = 'чем заняться осенью'

In [23]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['               совместно с Французским институтом моды и школой при Парижской палате высокой моды   разработал программу для начинающих дизайнеров  Об этом в понедельник  10 июля  сообщает        Первые студенты начнут обучение в сентябре  Продолжительность курса   полгода  он посвящен изучению инноваций и экологических аспектов моды  Теоретическая часть рассчитана на 25 дней занятий  остальное время будет посвящено стажировке в                К окончанию учебы каждый выпускник должен будет подготовить специальный проект  а затем представить его комиссии из преподавателей и сотрудников компании   Хранить наследие Дома и инвестировать в таланты   часть нашей долговременной стратегии  Я очень горжусь этим сотрудничеством     признался президент и исполнительный директор               Франческа Беллеттини  Дом моды               основан в 1962 году дизайнером Ивом Сен Лораном совместно с Пьером Берже  В 2002 году Сен Лоран принял решение отойти от дел и продал свой бренд холдингу       

In [24]:
get_response(TEXT, ft_index, modelFT, index_map)

[' АвтоВАЗ  снимет с конвейера автомобили      моделей                и         Им на смену придут другие   обновленные  сообщает во вторник  22 мая  принадлежащая семье Демьяна Кудрявцева газета  Ведомости   Как отмечает издание  в производственном плане завода на июль машин уже нет  Автопроизводитель поясняет  что прекращает производство дорестайлинговых моделей этих автомобилей и готовится к выпуску рестайлинговых в Х стиле                самый продаваемый автомобиль в серии  В 2009 и 2012 году она была самой популярной среди покупателей машиной в России  В апреле 2017 года количество выпущенных        достигло миллиона ',
 'В преддверие нового сезона парижский бутик         совместно с дизайн агентством              составил список всего необходимого  что может понадобиться во время посещения Недели моды  Об этом сообщает        В набор входят футболки от марки        с надписями           и             Дадут ли они беспрепятственно пройти за кулисы  не уточняется  Также на время д

In [25]:
TEXT = 'новости из мира искусственного интеллекта'

In [26]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['По мнению британского физика Стивена Хокинга стремительное развитие искусственного интеллекта представляет угрозу для человечества  пишет              Если сейчас люди разрабатывают компьютерные вирусы  то в будущем кто то сможет создать искусственный интеллект  который сможет улучшать и воспроизводить самого себя  Это станет новой формой жизни  которая превзойдет человека     заявил ученый  добавив  что машины в погоне за эффективностью могут пожелать избавиться от людей  Впрочем  если этого и не произойдет  люди рискуют погубить себя сами   Мы прошли точку невозврата  Земля становится слишком тесной для нас  население растет пугающими темпами     заключил физик  Избежать гибели  по мнению Хокинга  человечество может большим вовлечением людей в науку  чтобы получить возможность для начала колонизации других планет  26 октября сообщалось о создании самообучающегося искусственного интеллекта  способного обходить защиту сайтов от ботов ',
 'Предприниматель Илон Маск заявил  что искусст

In [27]:
get_response(TEXT, ft_index, modelFT, index_map)

['Российская студия разработчиков              раскрыла подробности об игре                  Об этом сообщил представитель компании                  является онлайн экшеном с элементами выживания в научно фантастическом сеттинге  Так  игроку предстоит исследовать локации  создавать броню и оружие  строить базы и сражаться с противниками  Разработчики обещают постоянно добавлять в игру новый контент  расширяя мир и историю игры   Мы стремимся создать живой и запоминающийся мир  который было бы интересно покорять как с друзьями  так и в одиночку    рассказал исполнительный продюсер проекта Владимир Макарычев    При разработке мы фокусируемся на предоставлении игрокам максимального разнообразия возможностей  чтобы и новички  и опытные любители жанра могли найти в                 для себя занятие по душе   Также в игре будет представлено несколько уникальных фракций с уникальной историей и способностями  В течение осени разработчики проведут закрытое альфа тестирование проекта  записаться 

Проверка работы показала, что чем менее абстрактен и более формализирован запрос, тем более лучшие результаты будут представлены